# Police Killings in Austin

This is a project for my data science blog [Quasi Qualified](https://quasiqualified.com/). The purpose of this project is to assess the statistics cited by the Wall Street Journal in a November 19, 2022, article about the killing of Rajan Moonesinghe in Austin, Texas. See that article [here](https://https://www.wsj.com/articles/entrepreneurs-death-at-hands-of-austin-police-prompts-calls-for-change-11671458081). Specifically, the hightlights what appears to be Austin's unique struggle with police killings. I assess three statistical claims.

Read my post [here](https://www.quasiqualified.com/posts/austin-police-killings/).

Primarily using [Plotly for Python](https://plotly.com/python/)

Very grateful to the data team at the Washington Post for their [Fatal Force Database](https://www.washingtonpost.com/graphics/investigations/police-shootings-database/). It's a lot of (thankless) work compiling and cleaning nationwide statistics about police killings. Their contribution helps further research and conversation about an alarming topic. As I mention at the end of my post, police violence, even when justified, should be avoided at all costs. The effort of the WaPo data team furthers that goal.

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
px.set_mapbox_access_token(open('C:/Users/thoma/OneDrive/Documents/data_science/mapbox-token-tpm.txt').read())
%pwd


'c:\\Users\\thoma\\OneDrive\\Documents\\data_science\\Projects\\data-police-shootings'

In [14]:
df = pd.read_csv("v2/fatal-police-shootings-data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8020 entries, 0 to 8019
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          8020 non-null   int64  
 1   date                        8020 non-null   object 
 2   threat_type                 8017 non-null   object 
 3   flee_status                 7047 non-null   object 
 4   armed_with                  7810 non-null   object 
 5   city                        8017 non-null   object 
 6   county                      3165 non-null   object 
 7   state                       8020 non-null   object 
 8   latitude                    7177 non-null   float64
 9   longitude                   7177 non-null   float64
 10  location_precision          7177 non-null   object 
 11  name                        7569 non-null   object 
 12  age                         7522 non-null   float64
 13  gender                      7988 

In [15]:
atx = df.loc[(df.city=='Austin') & (df.state=='TX')]
atx.race.replace({'H':'Hispanic','W':'White','B':'Black','A':'Asian'},inplace=True)
atx.date = pd.to_datetime(atx.date)
atx['incident_number'] = atx.reset_index().index + 1
atx.tail(5)

C:\Users\thoma\AppData\Local\Temp\ipykernel_9912\3699714675.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_9912\3699714675.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\thoma\AppData\Local\Temp\ipykernel_9912\3699714675.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

,id,date,threat_type,flee_status,armed_with,city,county,state,latitude,longitude,location_precision,name,age,gender,race,race_source,was_mental_illness_related,body_camera,agency_ids,incident_number
7471,8123,2022-06-12,threat,not,knife,Austin,NaN,TX,30.321218,-97.711923,not_available,Carlos Eli Chacon-Castillo,42.0,male,NaN,NaN,False,True,141,32
7477,8134,2022-06-14,undetermined,NaN,undetermined,Austin,NaN,TX,30.368548,-97.694425,not_available,Michael Christopher Carter,34.0,male,White,not_available,False,False,3492,33
7721,8684,2022-09-02,shoot,car,gun,Austin,NaN,TX,30.405121,-97.663901,not_available,Joshua Butler,44.0,male,NaN,NaN,False,False,3200,34
7784,8455,2022-09-23,threat,not,gun,Austin,NaN,TX,30.227891,-97.703363,not_available,Antonio Gonzales,29.0,male,Hispanic,not_available,True,False,141,35
7939,8595,2022-11-15,shoot,not,gun,Austin,NaN,TX,30.244151,-97.761640,not_available,Rajan David Moonesinghe,33.0,male,Asian,photo,False,False,141,36


In [16]:
fig = px.scatter_mapbox(atx, lat=atx.latitude, lon=atx.longitude, color=atx.race, mapbox_style='light', hover_name='name', zoom=10.5,
                        hover_data={'latitude': False, 'longitude': False, 'race': False, 'date': True, 'threat_type': True, 'was_mental_illness_related': True, 'flee_status': True, 'armed_with': True})
fig.update_layout(margin=dict(t=10, b=10, l=10, r=10), legend=dict(
    title='Race', xanchor='right', x=.99, yanchor='top', y=.95))
fig.update_traces(marker=dict(size=12))
fig.show(config={'displayModeBar': False})

# fig.write_html('charts/map.html',config={'displaylogo':False})


In [17]:
fig_scatter = px.scatter(atx, 'date', 'incident_number', hover_name='name', hover_data={'date': False, 'incident_number': False, 'was_mental_illness_related': False}, labels={
                         'incident_number': 'Total Police Killings Since 2015', 'date': None}, custom_data=['name','was_mental_illness_related'])
fig_scatter.update_layout(hovermode='x unified',
                          legend=dict(title='Mental Health Incident?', xanchor='left', x=.05),
                        )
fig_scatter.update_traces(fill='tozeroy', hovertemplate='<b>%{customdata[0]}</b><br>Count of Killings: %{y}<br>During Mental Crisis: %{customdata[1]}')

fig_scatter.show(config={'displaylogo': False})
# fig_scatter.write_html('charts/Austinkillings.html',config={'displaylogo':False})

In [18]:
states_dict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

In [26]:
# pivot for inter-city comparison
killings_all = pd.pivot_table(df, index=['city','state'], aggfunc='count').sort_values('agency_ids', ascending=False).reset_index()[['city','state','agency_ids']]
mental_all = pd.pivot_table(df.loc[(df.was_mental_illness_related==True)], index=['city','state'], aggfunc='count').sort_values('was_mental_illness_related', ascending=False).reset_index()[['city','state','was_mental_illness_related']]

# merge
killings_all = killings_all.merge(mental_all, on=['city','state'],how='outer').fillna(0)
killings_all.columns = ['city','state','killings','mental_incident']
killings_all.mental_incident = killings_all.mental_incident.astype(int)

# import and clean census data, population estimates from 2020
pop = pd.read_excel('populations.xlsx')
pop.columns = ['city', 'state', 'pop']
pop.city = pop.city.str[:-5]
pop.state = pop.state.str[1:]
pop = pop.replace({'state': states_dict})

# merge
killings_all = killings_all.merge(pop, on=['city','state'], how='left')
killings_all = killings_all.sort_values('pop', ascending=False).head(100)
killings_all['pop'] = killings_all['pop'].astype(int)

# generate per capita adjustments
killings_all['kpc'] = killings_all.killings / killings_all['pop']
killings_all['mhp'] = killings_all.mental_incident / killings_all.killings

# display 2015-present Mental Health Incident Killings per capita, 25 largest US cities
killings_all.head(25).sort_values('mhp', ascending = False).reset_index(drop=True)


,city,state,killings,mental_incident,pop,kpc,mhp
0,Boston,MA,10,4,674272,0.000015,0.400000
1,Austin,TX,36,14,963121,0.000037,0.388889
2,San Diego,CA,28,10,1385394,0.000020,0.357143
3,Las Vegas,NV,67,23,643292,0.000104,0.343284
4,San Jose,CA,24,8,1010908,0.000024,0.333333
5,Charlotte,NC,22,7,876747,0.000025,0.318182
6,Portland,OR,25,7,652388,0.000038,0.280000
7,Nashville,TN,15,4,689248,0.000022,0.266667
8,San Francisco,CA,20,5,870014,0.000023,0.250000
9,New York,NY,43,10,8772978,0.000005,0.232558


In [27]:
# display 2015-present total killings per capita, 25 largest US cities

killings_all.head(25).sort_values('kpc', ascending = False).reset_index(drop=True)

,city,state,killings,mental_incident,pop,kpc,mhp
0,Las Vegas,NV,67,23,643292,0.000104,0.343284
1,Denver,CO,47,3,717630,0.000065,0.063830
2,Phoenix,AZ,104,15,1611345,0.000065,0.144231
3,Oklahoma City,OK,41,9,682760,0.000060,0.219512
4,San Antonio,TX,68,8,1438227,0.000047,0.117647
5,Columbus,OH,40,2,905860,0.000044,0.050000
6,Jacksonville,FL,40,9,950463,0.000042,0.225000
7,Houston,TX,90,16,2300027,0.000039,0.177778
8,Portland,OR,25,7,652388,0.000038,0.280000
9,Austin,TX,36,14,963121,0.000037,0.388889


In [25]:
# recalculate tables, isolating from 2017-present to match WSJ claims.
killings_wsj = pd.pivot_table(df.loc[(df.date>'2017-12-01')], index=['city','state'], aggfunc='count').sort_values('agency_ids', ascending=False).reset_index()[['city','state','agency_ids']]
mental_wsj = pd.pivot_table(df.loc[((df.date>'2017-12-01')&(df.was_mental_illness_related==True))], index=['city','state'], aggfunc='count').sort_values('was_mental_illness_related', ascending=False).reset_index()[['city','state','was_mental_illness_related']]

# merge
killings_wsj = killings_wsj.merge(mental_wsj, on=['city','state'],how='outer').fillna(0)
killings_wsj.columns = ['city','state','killings','mental_incident']
killings_wsj.mental_incident = killings_wsj.mental_incident.astype(int)

# merge with population data
killings_wsj = killings_wsj.merge(pop, on=['city','state'], how='left')
killings_wsj = killings_wsj.sort_values('pop', ascending=False).head(100)

# generate variablesa
killings_wsj['pop'] = killings_wsj['pop'].astype(int)
killings_wsj['kpc'] = killings_wsj.killings / killings_wsj['pop']
killings_wsj['mhp'] = killings_wsj.mental_incident / killings_wsj.killings

# display killings per capita, 25 largest US cities, 2017-present
killings_wsj.head(25).sort_values(['kpc'], ascending = False).reset_index(drop=True)

,city,state,killings,mental_incident,pop,kpc,mhp
0,Las Vegas,NV,42,12,643292,0.000065,0.285714
1,Denver,CO,33,2,717630,0.000046,0.060606
2,Phoenix,AZ,70,6,1611345,0.000043,0.085714
3,Oklahoma City,OK,25,5,682760,0.000037,0.200000
4,San Antonio,TX,49,5,1438227,0.000034,0.102041
5,Portland,OR,19,4,652388,0.000029,0.210526
6,Jacksonville,FL,26,7,950463,0.000027,0.269231
7,Houston,TX,60,11,2300027,0.000026,0.183333
8,Columbus,OH,23,0,905860,0.000025,0.000000
9,Los Angeles,CA,86,9,3889834,0.000022,0.104651


In [24]:
# display mental health killings per capita, 25 largest US cities, 2017-present
killings_wsj.head(25).sort_values('mhp', ascending = False).reset_index(drop=True)

,city,state,killings,mental_incident,pop,kpc,mhp
0,San Diego,CA,18,7,1385394,0.000013,0.388889
1,Charlotte,NC,11,4,876747,0.000013,0.363636
2,Austin,TX,20,7,963121,0.000021,0.350000
3,Nashville,TN,10,3,689248,0.000015,0.300000
4,Las Vegas,NV,42,12,643292,0.000065,0.285714
5,San Francisco,CA,7,2,870014,0.000008,0.285714
6,Jacksonville,FL,26,7,950463,0.000027,0.269231
7,Portland,OR,19,4,652388,0.000029,0.210526
8,Boston,MA,5,1,674272,0.000007,0.200000
9,Oklahoma City,OK,25,5,682760,0.000037,0.200000


In [23]:
# generate scatter plot with OLS regression line, dropping New York
fig_kills = px.scatter(killings_all.drop(index=9), x='pop',y='killings',hover_name='city', log_x=True, trendline='ols', color_discrete_map={'963121':'r'}, labels={'pop':'Population','killings':'Police Killings Since 2015'})

# add Austin with distinct marker
fig_kills.add_trace(go.Scatter(x= [963121], y = [36], mode='markers', marker_size=15, name='Austin', marker_symbol='cross', hovertemplate='<b>Austin, Texas</b><br>Population: %{x}<br>Police Killings: %{y}'))
fig_kills.update_layout(legend=dict(title=None, xanchor='left', x=.05))

# add New York in red
fig_kills.add_trace(go.Scatter(x= [8772978], y = [43], mode='markers', name='New York', marker_symbol='circle', hovertemplate='<b>New York, NY</b><br>Population: %{x}<br>Police Killings: %{y}', marker_color='red', showlegend=False))

fig_kills
# fig_kills.write_html('charts/cityscatter.html', config= {'displaylogo': False})